In [10]:
import tensorflow
import torch
import numpy as np
import cv2

In [116]:
model = tensorflow.keras.models.load_model("D:/CVX/Cervix_Cancer_Classification/densenet_cvx.h5")

In [11]:
mapping_dict = {0:'Type 1',1:'Type 2', 2: 'Type 3'}

In [134]:
img = cv2.imread("D:/Project_CVX_Classification/dataset/test/Type_1/2876.jpg")

In [135]:
img = (cv2.resize(img, (224,224)))/255

In [136]:
img = np.expand_dims(img,axis= 0)

In [137]:
probs = model.predict(img)[0]

1/1 [==============================] - 0s 150ms/step


In [139]:
np.argmax(probs)

0

In [143]:
probs[2]

0.00024564218

In [140]:
probs[0]

0.94693935

In [97]:
import numpy as np
import pandas as pd
import glob
import cv2
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.losses import mean_squared_error
from tensorflow.keras.utils import to_categorical
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import classification_report
from tqdm import tqdm
# from imgaug import augmenters
import matplotlib.gridspec as gridspec
from PIL import Image
import os
import json
import glob
import time
import datetime
from collections import Counter
from concurrent import futures
import threading
import matplotlib.pyplot as plt
%matplotlib inline
# from sklearn.preprocessing import LabelEncoder
from prettytable import PrettyTable
import zipfile
import seaborn as sns
# from numba import cuda

In [114]:
from sklearn.preprocessing import LabelEncoder

In [98]:
test_base_dir = os.path.join('D:/Project_CVX_Classification/dataset/test')
type1_test_dir = os.path.join(test_base_dir,'Type_1')
type2_test_dir = os.path.join(test_base_dir,'Type_2')
type3_test_dir = os.path.join(test_base_dir,'Type_3')

type1_test_files = glob.glob(type1_test_dir+'/*.jpg')
type2_test_files = glob.glob(type2_test_dir+'/*.jpg')
type3_test_files = glob.glob(type3_test_dir+'/*.jpg')

len(type1_test_files),len(type2_test_files),len(type3_test_files)

(200, 200, 200)

In [127]:
#Building a dataframe mapping images and Cancer type
test_files_df = pd.DataFrame({
    'filename': type1_test_files + type2_test_files + type3_test_files,
    'label': ['Type_1'] * len(type1_test_files) + ['Type_2'] * len(type2_test_files) + ['Type_3'] * len(type3_test_files)
}).sample(frac=1, random_state=42).reset_index(drop=True)


In [128]:
test_files_df

,filename,label
0,D:/Project_CVX_Classification/dataset/test\Typ...,Type_1
1,D:/Project_CVX_Classification/dataset/test\Typ...,Type_3
2,D:/Project_CVX_Classification/dataset/test\Typ...,Type_3
3,D:/Project_CVX_Classification/dataset/test\Typ...,Type_1
4,D:/Project_CVX_Classification/dataset/test\Typ...,Type_1
...,...,...
595,D:/Project_CVX_Classification/dataset/test\Typ...,Type_1
596,D:/Project_CVX_Classification/dataset/test\Typ...,Type_1
597,D:/Project_CVX_Classification/dataset/test\Typ...,Type_2
598,D:/Project_CVX_Classification/dataset/test\Typ...,Type_3


In [129]:
test_array = []
test_labels = []
IMG_DIMS = (224, 224)

for i in tqdm(range(len(test_files_df))):
    img_path = test_files_df['filename'][i]
    img = cv2.imread(img_path)
    img = cv2.resize(img, dsize=IMG_DIMS,
                     interpolation=cv2.INTER_CUBIC)
    img = np.array(img, dtype=np.float32)
    test_array.append(img)
    test_labels.append(test_files_df['label'][i])


100%|██████████| 600/600 [00:01<00:00, 309.38it/s]


In [133]:
test_files_df['filename'][0]

'D:/Project_CVX_Classification/dataset/test\\Type_1\\35 (2).jpg'

In [130]:
#scaling the test set and one-hot encoding the test labels
test_labels= np.array(test_labels)
test_imgs_scaled = np.array(test_array) / 255.
test_imgs_scaled.shape, test_labels.shape

le = LabelEncoder()
le.fit(test_labels)
test_labels_enc = le.transform(test_labels)

test_labels_1hotenc = to_categorical(test_labels_enc, num_classes=3)


print(test_labels[:6], test_labels_enc[:6])
print(test_labels[:6], test_labels_1hotenc[:6])

['Type_1' 'Type_3' 'Type_3' 'Type_1' 'Type_1' 'Type_2'] [0 2 2 0 0 1]
['Type_1' 'Type_3' 'Type_3' 'Type_1' 'Type_1' 'Type_2'] [[1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]]


In [117]:
# Predictions and Confusion Matrix
predictions_densenet = model.predict(test_imgs_scaled)
predicted_labels_densenet = np.argmax(predictions_densenet, axis=1)
true_labels_densenet = np.argmax(test_labels_1hotenc, axis=1)


19/19 [==============================] - 58s 3s/step


In [121]:
from sklearn.metrics import confusion_matrix

In [122]:
conf_mat_densenet = confusion_matrix(true_labels_densenet, predicted_labels_densenet)

In [123]:
conf_mat_densenet

array([[115,  76,   9],
       [ 10, 169,  21],
       [  3,  91, 106]], dtype=int64)